# Formation RNAseq CEA - juin 2024

*Enseignantes: Sandrine Caburet et Claire Vandiedonck*

Session IFB : 5 CPU + 21 GB de RAM

# Part 6: Read Counts

   

- 0. 1 - About session for IFB core cluster
- 0. 2 - Parameters to be set or modified by the user
- 1 - Gene level quantification using ``featureCounts``
- 2 - Pseudo-mapping with Salmon
- 3 - Monitoring disk usage

- This notebook contains heavy running cells in Section 2, so make sure before you launch these cells that they are adapted to your environment. <br>
For Samtools, the ``-m`` option is the **RAM memory size that will be used by each thread**! <br>
<blockquote>
        See options <code>--threads 3 -m 4G</code> in Samtools line, Section 2  <br>
        Adapt <code>-T 4</code> in featureCounts lines to set it to 70-80% of available CPU number. <br>
        Adapt <code>-s 0</code> in  line to fit your library preparation. In the current notebook version, this option is set to <code>0</code> as the librairies are unstranded. <br>
</blockquote>

<div class="alert alert-block alert-danger">
    <b>Values set in this notebook are valid for a 5-CPU session with access to 21 GB of RAM</b>. Ideally, use 70-80% of the CPU amount your system or session has. <br>
    DO NOT ask for more RAM than you can use.
</div>

---
---
## 0. Before going further
---

<div class="alert alert-block alert-danger"><b>Caution:</b> 
Before starting the analysis, save a backup copy of this notebok : in the left-hand panel, right-click on this file and select "Duplicate"<br>
You can also make backups during the analysis. Don't forget to save your notebook regularly: <kbd>Ctrl</kbd> + <kbd>S</kbd> or click on the 💾 icon.
</div>

---

## 0.1 - About session for IFB core cluster

<em>loaded JupyterLab</em> : Version 3.5.0

In [1]:
## Code cell 1 ##

echo "=== Cell launched on $(date) ==="
squeue -hu $USER 

echo "=== Current IFB session size: as an indication: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ==="
jobid=$(squeue -hu $USER | awk '/sys/dash {print $1}')

sacct --format=JobID,AllocCPUS,ReqMem,NodeList,Elapsed,State --jobs ${jobid}

=== Cell launched on Tue Jun 18 20:44:27 CEST 2024 ===
          40262071      fast sys/dash cvandied  R    3:44:53      1 cpu-node-51
=== Current IFB session size: as an indication: Medium (4CPU, 10GB) or Large (10CPU, 50GB) ===
JobID         AllocCPUS     ReqMem        NodeList    Elapsed      State 
------------ ---------- ---------- --------------- ---------- ---------- 
40262071              5        21G     cpu-node-51   03:44:53    RUNNING 
40262071.ba+          5                cpu-node-51   03:44:53    RUNNING 


In [2]:
## Code cell 2 ##

module load samtools/1.18 subread/2.0.6 salmon/1.10.2 multiqc

# module load samtools/1.10 subread/2.0.1 in 2023

echo "===== bam sorting by names ====="
samtools --version | head -n 2
echo "===== gene level quantification ====="
featureCounts -v
echo "===== Pseudo mapping and quantification with Salmon ====="
salmon -v
echo "===== quality reports compilation ====="
multiqc --version

===== bam sorting by names =====
samtools 1.18
Using htslib 1.19
===== gene level quantification =====

featureCounts v2.0.6

===== Pseudo mapping and quantification with Salmon =====
salmon 1.10.2
===== quality reports compilation =====
multiqc, version 1.13


---

## 0.2 - General parameters to be set or modified by the user


- Precise the **maximum amount of CPU** (central processing units, cores) and **RAM-memory (in Bytes)** that programs can use.<a id="computressources"></a>

In [3]:
## Code cell 3 ##

authorizedCPU=4           # 4 CPU

authorizedRAM=20000000000  # 20GB

- Using a full path with a `/` at the end, **define the folder** where you want or have to work with `gohome` variable:

In [4]:
## Code cell 4 ##

gohome="/shared/projects/2413_rnaseq_cea/"

echo "=== Home root folder is ==="
echo "${gohome}"
echo ""
echo "=== Working (personal) folder tree ==="
tree -d -L 2 "${gohome}$USER"
echo "=== current working directory ==="
echo "${PWD}"

=== Home root folder is ===
/shared/projects/2413_rnaseq_cea/

=== Working (personal) folder tree ===
/shared/projects/2413_rnaseq_cea/cvandiedonck
|-- Data
|   `-- fastq
|-- Results
|   |-- fastp
|   |-- fastq_screen
|   |-- fastqc
|   |-- logfiles
|   |-- multiqc
|   |-- qualimap
|   |-- samtools
|   `-- star
|-- archives
|-- ongoing
|-- run_notebooks
|-- temporary
|   |-- Results
|   `-- run_notebooks
`-- tmp

18 directories
=== current working directory ===
/shared/ifbstor1/projects/2413_rnaseq_cea/cvandiedonck


- ... and remember the folder for log files.

In [5]:
## Code cell 5 ##

logfolder="${gohome}$USER/Results/logfiles/"
echo "the folder for log files is ${logfolder}"

the folder for log files is /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/logfiles/


---
---
## 1 - Gene level quantification using <code>featureCounts</code>
---

### 1.1- Tool presentation
---

`featureCounts` is part of the package <a href="http://subread.sourceforge.net/">SubRead</a>, to be used with bash, and RSubRead, to be used with R. This tool allows to attribute mapped reads to their matching feature (exon, gene, promoter, ...) on the genome and summarize counts per feature.  
  
In SubRead user guide, developpers recommend to use <i>specialized transcript-level quantification tools [...] for counting reads to transcripts</i> (see section 6.2.5, page 34 of pdf manual you can download with this <a href="https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwie1YDqwpD_AhXgU6QEHVLnB14QFnoECAsQAQ&url=https%3A%2F%2Fbioconductor.org%2Fpackages%2Frelease%2Fbioc%2Fvignettes%2FRsubread%2Finst%2Fdoc%2FSubreadUsersGuide.pdf&usg=AOvVaw1b3PpVhTNokdJHARtYAXgf">link</a>). So we will only generate gene-level quantification with `featureCounts`.

In [6]:
## Code cell 6 ##

featureCounts -v


featureCounts v2.0.6



A simple usage to count paired-end sequencing data, then counts fragments instead or reads, is:  
<code>featureCounts -p -a annotation.gtf \ 
                  -o counts.txt \ 
                  alignment.bam</code>

The main <code>featureCounts</code> options correspond to: <br>
- `-p` to count fragments instead or reads (paired-end data)
- `-a TEXT` to locate annotation file, in GTF/GFF format by default (<code>-f</code> to be used to give other file format) that can be a <code>.gzip</code> one. <br>
- `-o TEXT` to set filename for counts<br>
- then alignement files (it may be a list) come. Either in BAM or SAM format, they can be sorted by read names or by chromosomal coordinates.
- `M`: By default, multi-mapped reads (or fragments) are not considered unless we use `-M` option (see others parameters to set in manual, pdf pages 38 to 43).
- `-T exon` *(by default)*: It will select exon lines in annotation file to attribute reads (or fragments).
- `-g gene_id` *(by default)*: Then, it will count them according to gene_id meta-feature level.

We will use some other options:  

- <code>-s INTEGER</code>, to specify strandness. Possible values include: 0 (unstranded), 1 (stranded) and 2 (reversely stranded). Default value is 0. <br>
- <code>-T INTEGER</code>, to set the number of threads to use (default, 1) <br>
- <code>--verbose</code>, to get information for debugging, such as unmatched chromosome/contig names. <br>
    
As temporary files are saved by default to directory specified in <code>-o</code> option, we won't use <code>--tmpDir STRING</code> option <br>
</blockquote>

For paired-end data, it is possible to ask for filtering fragments that have both ends aligned (`-B` option), on same chromose and strand (`-C`) and even separated by an insert distance (`-P`, included in `-d` and `-D` values).  

Besides, input files will be used as sorted by names. Even if ***featureCount*** handles `.bam` files as fast as ***samtools*** does, we will nonetheless use samtools and `--donotsort` option. Indeed, some files may be bigger than supported by featureCounts.

---
### 1.2- Step preparation
---

- With a `/` at the end, precise the folder where alignment ``_ALigned.SortedByCoord.out.bam`` files produced by ***Star*** are:

In [7]:
## Code cell 7 ##

mappedfolder="${gohome}$USER/Results/star/"
echo "There are $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam | wc -l) bam files:"
ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam

There are 3 bam files:
/shared/projects/2413_rnaseq_cea/cvandiedonck/Results/star/SRR12730409_Aligned.sortedByCoord.out.bam
/shared/projects/2413_rnaseq_cea/cvandiedonck/Results/star/SRR12730410_Aligned.sortedByCoord.out.bam
/shared/projects/2413_rnaseq_cea/cvandiedonck/Results/star/SRR12730411_Aligned.sortedByCoord.out.bam


- Please remember the path to the uncompressed annotation file, especially if you changed it in previous notebooks. We can also show its first line to verify the version (here we work with the M35 release on the GRCm39 mouse genome)

In [8]:
## Code cell 8 ##

gtffile="${gohome}alldata/Reference/extracted/genome_annotation-M35.gtf"
echo "The transcript reference gtf file is ${gtffile}"
echo "First rows of the annotation file: ${gtffile}"
head -n 6 ${gtffile}

The transcript reference gtf file is /shared/projects/2413_rnaseq_cea/alldata/Reference/extracted/genome_annotation-M35.gtf
First rows of the annotation file: /shared/projects/2413_rnaseq_cea/alldata/Reference/extracted/genome_annotation-M35.gtf
##description: evidence-based annotation of the mouse genome (GRCm39), version M35 (Ensembl 112)
##provider: GENCODE
##contact: gencode-help@ebi.ac.uk
##format: gtf
##date: 2024-02-27
chr1	HAVANA	gene	3143476	3144545	.	+	.	gene_id "ENSMUSG00000102693.2"; gene_type "TEC"; gene_name "4933401J01Rik"; level 2; mgi_id "MGI:1918292"; havana_gene "OTTMUSG00000049935.1";


- We need to create a destination folder...

In [9]:
## Code cell 9 ##

featcountfolder="${gohome}$USER/Results/featurecounts/"
mkdir -p ${featcountfolder}
tree -d -L 1 "${gohome}$USER/Results/"

/shared/projects/2413_rnaseq_cea/cvandiedonck/Results/
|-- fastp
|-- fastq_screen
|-- fastqc
|-- featurecounts
|-- logfiles
|-- multiqc
|-- qualimap
|-- samtools
`-- star

9 directories



---

### 1.3- Compute reads or fragments counts
---

#### 1.3.1 - Running <code>featuresCounts</code> on individual samples
---

- Before you run the following cells, make sure that they are adapted to your environment. <br>
For Samtools, the ``-m`` option is the **RAM memory size that will be used by each thread**! <br>
<blockquote>
        See options <code>--threads 3 -m 5G</code> in Samtools line <br>
        Adapt <code>-T 4</code> in featureCounts lines to set it to 70-80% of available CPU number. <br>
        Adapt <code>-s 0</code> in  line to fit your library preparation. In the current notebook version, this option is set to <code>0</code> as the librairies are unstranded. <br>
</blockquote>

<div class="alert alert-block alert-danger">
    <b>Values set in this notebook are valid for a 5-CPU session with access to 21 GB of RAM</b>. Ideally, use 70-80% of the CPU amount your system or session has. <br>
    DO NOT ask for more RAM than you can use.
</div>

<div class="alert alert-block alert-warning">
    <b><code>-T</code> option in featureCounts command line doesn't have an impact during counting process</b>. It has an effect when BAM are sorted on the fly by features counts, but this is not the case here, as the input file is sorted before by samtools.
</div>

In [10]:
## Code cell 10 ##

logfile="${logfolder}featureCounts-gene-level-counts-individualsamples_samtoolsSort.log"
echo "Screen output is redirected to ${logfile}"

Screen output is redirected to /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/logfiles/featureCounts-gene-level-counts-individualsamples_samtoolsSort.log


In the next cell, each sample file is analysed individually: first <code>samtools</code> generates a bam sorted by read names, that is used immediately by <code>featureCounts</code>. The intermediate bam file is then removed.   
Therefore we obtain one count file per sample. 

In [11]:
## Code cell 11 ##

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

time for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do  
    
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "===== Processing sampleID: ${id}..." | tee -a ${logfile}
    
    # outputfiles
    mytempfile="${featcountfolder}${id}_Aligned.sortedByNames.bam"
    myoutfile="${featcountfolder}${id}_paired-unstranded"
    
    # bam sorting...
    echo "samtools starts at $(date)" >> ${logfile}
    samtools sort -n \
                --threads 3 -m 4G \
                --output-fmt BAM \
                -o ${mytempfile} \
                -T ${featcountfolder} \
                ${fn} \
                &>> ${logfile}
    echo "samtools ends at $(date)" >> ${logfile}

    # some user conversation to help being patient
    echo "...changing tool..." | tee -a ${logfile}

    # then featureCounts
    echo "featureCounts starts at $(date)" >> ${logfile}

    featureCounts -p -s 0 -T 4 \
                  -a "${gtffile}" \
                  -o "${myoutfile}.counts" \
                  ${mytempfile} \
                  --donotsort \
                  --verbose \
                  &>> ${logfile}
    echo "featureCounts ends at $(date)" >> ${logfile}
    
    # removing extra bam file... saving disk space
    rm ${mytempfile}
    
    echo "... done" | tee -a ${logfile}
    
done

===== Processing sampleID: SRR12730409...
...changing tool...
... done
===== Processing sampleID: SRR12730410...
...changing tool...
... done
===== Processing sampleID: SRR12730411...
...changing tool...
... done

real	13m16.666s
user	40m22.400s
sys	0m57.766s


In [12]:
## Code cell 12 ##

echo "operation ends at $(date)" >> ${logfile}

echo "=== Files created after featureCounts ===" >> ${logfile}
ls -lh "${featcountfolder}" >> ${logfile}
echo "featureCounts generated $(ls "${featcountfolder}"*.counts | wc -l) count files." \
    | tee -a ${logfile}
echo "featureCounts generated $(ls "${featcountfolder}"*.counts.summary | wc -l) summary files." \
    | tee -a ${logfile}

featureCounts generated 3 count files.
featureCounts generated 3 summary files.


#### 1.3.2 - Running <code>featuresCounts</code> on multiple samples
---

The above cell processed each sample file individually: each sorted bam generated by <code>samtools</code> is used as input by <code>featureCounts</code>, and we obtain one count file per sample. Those count files could be later joined in a single count matrix for the next step of analysis.   

Alternatively, <code>featuresCounts</code> can handle multiple bam files at once, creating directly a single count matrix for all the samples.    
This requires that all the sorted bam files are available as input for <code>featuresCounts</code>.    
This was done beforehand for all 11 samples, by running the next 2 cells.   
Code cell 14 runs <code>samtools</code> to generate (and keep!) the sorted bam files.   
Code cell 16 runs <code>featuresCounts</code> only once, using the list of sorted bam filenames as input.  

To run those cells in your own project, simply change their type from *Raw* to *Code*.


We can see the proportions of alignements that match a single gene (if default `featureCounts` parameters were kept) in the logfile and a research for specified patterns (`grep -e PATTERN`).

Let's have a look at the beginning of result files from featureCounts.   
We retrieve a copy of the count for all samples in our own Results/featurecounts folder, and we display the two types of counts files.

In [13]:
## Code cell 19 ##

cp ${gohome}alldata/Results/featurecounts/11samples_* ${gohome}$USER/Results/featurecounts/

head -n 8 ${gohome}$USER/Results/featurecounts/*_paired-unstranded.counts

==> /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/featurecounts/11samples_paired-unstranded.counts <==
# Program:featureCounts v2.0.6; Command:"featureCounts" "-p" "-s" "0" "-T" "16" "-a" "/shared/projects/2413_rnaseq_cea/alldata/Reference/extracted/genome_annotation-M35.gtf" "-o" "/shared/projects/2413_rnaseq_cea/alldata/Results/featurecounts/11samples_paired-unstranded.counts" "/shared/projects/2413_rnaseq_cea/alldata/Results/featurecounts/SRR12730403_Aligned.sortedByNames.bam" "/shared/projects/2413_rnaseq_cea/alldata/Results/featurecounts/SRR12730404_Aligned.sortedByNames.bam" "/shared/projects/2413_rnaseq_cea/alldata/Results/featurecounts/SRR12730405_Aligned.sortedByNames.bam" "/shared/projects/2413_rnaseq_cea/alldata/Results/featurecounts/SRR12730406_Aligned.sortedByNames.bam" "/shared/projects/2413_rnaseq_cea/alldata/Results/featurecounts/SRR12730407_Aligned.sortedByNames.bam" "/shared/projects/2413_rnaseq_cea/alldata/Results/featurecounts/SRR12730408_Aligned.sortedByNam

As you can see, the genes are in rows and the counts in the samples are provided in the one-before-last column.   

---
---
## 2 - Pseudo-mapping with **Salmon**
---

### 2.1- Tool presentation
---

The **pseudomapping** or **pseudoalignement** is an alternative method of counting reads to transcripts and genes without prior mapping to a reference genome. It relies on prior indexation of all transcripts (rather than indexing the genome). Using theses indeces, reads are directly assigned to transcripts. This method does not generate BAM/SAM alignement files. It produces directly the matrix of counts for each transcript.

The main advantages of this method is the speed and direct count of all transcript isoforms (only annotated ones). On the opposite, only annotated transcripts can be counted and it does not allow the discovery of new transcripts. It is also impossible to vizualize the alignments of the reads with visualization tools such as IGV.
To quantify expression at the gene level, transcript counts can simply be aggregated (i.e summed) by genes.

Two softwares are currently available with very similar characteristics and outputs, namely [Salmon](https://salmon.readthedocs.io/en/latest/index.html#) and [Kallisto](https://pachterlab.github.io/kallisto/). You can find a brief summary of both methods here: https://learn.gencore.bio.nyu.edu/rna-seq-analysis/salmon-kallisto-rapid-transcript-quantification-for-rna-seq-data/. Both methods are higly similar and provide same results (see blog of Lior Pachter: https://liorpachter.wordpress.com/2015/05/10/near-optimal-rna-seq-quantification-with-kallisto/ claming that Salmon copied Kallisto). Nonetheless, Salmon is somehow easier to use., soo we will use Salmon in this pipeline.

To use **Salmon** (documentation here: https://salmon.readthedocs.io/en/latest/index.html), two main steps are necessary:

**1. build the ***gentrome*** index**:
Gentrome is a contraction for genome and transcriptome. The `gentrome.fa` is a new hybrid fasta file which contains the decoy sequences from the genome, concatenated with the transcriptome, following the indications provided by the annotation file.

The basic command to generate the index on the gentrome sequence is:
<code> salmon index -t gentrome.fa.gz -i salmon_annot_index --gencode</code>

**2. run pseudo-alignement of reads** :

The generic command is:

salmon quant -i <salmon_index> -l <lib_type> -1 <read_1.fastq> -2 <read_2.fastq> -o <output_dir>

<code>salmon quant -i salmon_annot_index \
         -l A \
         -1 sampleID_R1_fastp.fastq.gz \
         -2 sampleID_R2_fastp.fastq.gz \
	 --validateMappings \
	 -o output_dir</code>
     
The main options above are:
- `-i` : to specify the ngentrome index file
- `-l` : to specify the library type with the argument `A` to let Salmon infer it, or manually specifying the orientation (I = inward, O = outward, M = matching), the strandedness (S = stranded, U = unstranded) and the strand from which the read originates in a strand-specific protocol (F = read 1 comes from the forward strand, R = read 1 comes from the reverse strand)

Among other options that can be used, we higlight two of them:
- `-p` : to specify the number of threads, we will use it.
- `--numBootstraps` : to optionally compute bootstrapped abundance estimates. This is very time consuming as it is done by resampling (with replacement) from the counts assigned to the fragment equivalence classes, and then re-running the optimization procedure,for each such sample. It is used to assess technical variance in the main abundance estimates. The more samples computed, the better the estimates of variance, but the more computation (and time) required. We are not using it.



For each sample, we obtain a folder in salmon _res_folder with `quant.sf` as main output. This file contains ***TPM (transcripts per million)*** and read count for each transcript ID in rows.

<div class="alert alert-block alert-danger">Caution, TPM are not integger values. It may have implications for downstream analyses.</div>

---
### 2.2- Step preparation
---

- We check the version of Salmon we are using:

In [14]:
## Code cell 20 ##

salmon -v

salmon 1.10.2


- We precise the folder where the Salmon pseudo-alignments will be saved:

In [15]:
## Code cell 21 ##

salmonfolder="${gohome}$USER/Results/salmon/"
mkdir -p ${salmonfolder}
echo "The resulting salmon files will be in ${salmonfolder}"

The resulting salmon files will be in /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/salmon/


- and for Salmon pseudomapping, we will also need to generate a folder with the indexing of the transcripts, so we remember the path `Reference` including all reference sequences and annotations (we use the common file in the `alldata` folder):

In [16]:
## Code cell 22 ##

reffolder="${gohome}alldata/Reference/"
# mkdir -p ${reffolder}
echo "The general folder for reference genome, annotations and index files is ${reffolder}"


salmonreffolder="${reffolder}/salmon/"
# mkdir -p ${salmonreffolder}
echo "The index folder for Salmon is ${salmonreffolder}"

The general folder for reference genome, annotations and index files is /shared/projects/2413_rnaseq_cea/alldata/Reference/
The index folder for Salmon is /shared/projects/2413_rnaseq_cea/alldata/Reference//salmon/


- finally, for Salmon pseudomapping, we will also need to specify the path to our input files, i.e, the fastq files after fastp quality control:

In [17]:
## Code cell 23 ##

fastpfolder="${gohome}$USER/Results/fastp/"
echo "The analysed fastp.fastq files are in ${fastpfolder}"
echo "and they are:"
ls ${fastpfolder} | grep -v -e "_removed" 

#| wc -l

The analysed fastp.fastq files are in /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/fastp/
and they are:
SRR12730409_1.fastp.fastq.gz
SRR12730409_2.fastp.fastq.gz
SRR12730409_fastp.html
SRR12730409_fastp.json
SRR12730410_1.fastp.fastq.gz
SRR12730410_2.fastp.fastq.gz
SRR12730410_fastp.html
SRR12730410_fastp.json
SRR12730411_1.fastp.fastq.gz
SRR12730411_2.fastp.fastq.gz
SRR12730411_fastp.html
SRR12730411_fastp.json


- an of course, we do not forget the log file !

In [18]:
## Code cell 24 ##

logfile4="${logfolder}Salmon.log"
echo "Screen output is redirected to ${logfile4}"

Screen output is redirected to /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/logfiles/Salmon.log


---
### 2.3- Run Salmon
---

#### 2.3.1. Build the gentrome index


To gain time and preserve space, we already performed this step and the results are available for eveyone in the alldata folder.

- obtain the **transcriptome sequences**


- obtain the **genome sequence** *(already done in Pipe 04)*

- **concatenate transcript sequences and genome sequence** :

- extract the **name of contigs of the genome sequence** (headers of fasta sequences)

Salmon recommands to use a selective alignment with a decoy-aware transcriptome, to mitigate potential spurious mapping of reads that actually arise from some unannotated genomic locus that is sequence-similar to an annotated transcriptome. Thus, decoy sequences will improve the quality of the pseudomammping.

There are actually two methods for generating decoyed sequences, as specified in the documentation of Salmon. Here we use the second one, meaning the entire genome of the organism is used as the decoy sequence.

- **transcriptome indexation** with the basic command (for mouse genome version 39):

We choose the kmer size of 31 as recommanded in Salmon documentation for reads of at least 75 bases.

#### 2.3.1. Run the pseudoalignment

In [19]:
## Code cell 31 ##

echo "Screen output is redirected to ${logfile4}"

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile4}
echo "before loop"

time for read1 in $(ls "${gohome}$USER/Results/fastp/"*_1.fastp.fastq.gz); do
    echo "starting the loop with ${read1}"
    
    # handling names with the sample name
    samplenum=$(basename ${read1} | cut -d"_" -f1)
    echo "====== Processing sampleID: ${samplenum}..." | tee -a ${logfile4}
    read2=$(echo ${read1} | sed 's#_1#_2#')

    echo "Salmon starts at $(date)" >> ${logfile4}
    
    # Salmon working
    salmon quant -i "${salmonreffolder}salmon_vM35_index" -l A \
         -1 ${read1} \
         -2 ${read2} \
         --validateMappings \
         --threads ${authorizedCPU} -o "${salmonfolder}${samplenum}" \
         |& tee -a ${logfile4}
         
    echo "Salmon ends at $(date)" >> ${logfile4}
    
    echo "...done" | tee -a ${logfile4} 
done  

Screen output is redirected to /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/logfiles/Salmon.log
before loop
starting the loop with /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/fastp/SRR12730409_1.fastp.fastq.gz
====== Processing sampleID: SRR12730409...
Version Server Response: Not Found
### salmon (selective-alignment-based) v1.10.2
### [ program ] => salmon 
### [ command ] => quant 
### [ index ] => { /shared/projects/2413_rnaseq_cea/alldata/Reference//salmon/salmon_vM35_index }
### [ libType ] => { A }
### [ mates1 ] => { /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/fastp/SRR12730409_1.fastp.fastq.gz }
### [ mates2 ] => { /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/fastp/SRR12730409_2.fastp.fastq.gz }
### [ validateMappings ] => { }
### [ threads ] => { 4 }
### [ output ] => { /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/salmon/SRR12730409 }
Logs will be written to /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/salmon/SRR12730409

For more details, you can follow a nice tutorial here: https://combine-lab.github.io/alevin-tutorial/2019/selective-alignment/ or this one: https://combine-lab.github.io/salmon/getting_started/

---
---
## 3 - MultiQC on featureCounts and Salmon
---

### **3.1 - Folder, filename, title and comment**

Let's indicate where report files are to be placed:

In [26]:
## Code cell 39 ## 

qcsummaries="${gohome}$USER/Results/multiqc/"

We specify then names for files and title to display on html report page. <a id="multiqctextvar2"></a>

In [27]:
## Code cell 40 ## 

inamemyfile="5_featureCounts_salmon_3samples"

mytitle=$(echo "Quality check after featureCounts and Salmon")

To keep record of what have been done with these files, we add an comment to remember for later use and for informing others readers:

In [28]:
## Code cell 41 ## 

mycomment=$(echo "featureCounts run at gene level and Salmon for pseudomapping on transcripts using the mouse genome (GRCm39), version M35 (Ensembl 112)")

### **3.2 - Generate summary report**

In [36]:
## Code cell 42 ## 

logfile5="${logfolder}multiqc-featurecounts-salmon.log"
echo "Screen output is also saved in ${logfile5}"

echo "operation starting by $(date)" >> ${logfile5}
multiqc --interactive --export \
        --module featureCounts ${featcountfolder} \
        --module salmon ${salmonfolder} \
        --outdir "$qcsummaries" \
        --filename "${inamemyfile}" \
        --title "${mytitle}"  \
        --comment "${mycomment}" \
        "${gohome}$USER/Results/" \
        |& tee -a ${logfile}
echo "operation finished by $(date)" >> ${logfile5}

Screen output is also saved in /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/logfiles/multiqc-featurecounts-salmon.log

  /// MultiQC 🔍 | v1.13

|           multiqc | MultiQC Version v1.22.2 now available!
|           multiqc | Report title: Quality check after featureCounts and Salmon
|           multiqc | Only using modules: featureCounts, salmon
|           multiqc | Search path : /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/featurecounts
|           multiqc | Search path : /shared/projects/2413_rnaseq_cea/cvandiedonck/Results/salmon
|           multiqc | Search path : /shared/projects/2413_rnaseq_cea/cvandiedonck/Results
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 775/775  
|    feature_counts | Found 11 reports
|            salmon | Found 3 meta reports
|            salmon | Found 3 fragment length distributions
|           multiqc | Compressing plot data
|           multiqc | Report      : ../../../../projects/2413_rnaseq_cea/cvandiedonck/

In [25]:
## Code cell 43 ## 

# to see which files we have afterward and follow folder sizes
ls -lh "${qcsummaries}" >> ${logfile}
ls -lh "${gohome}$USER/Results/" >> ${logfile}

---
---
## 4 - Monitoring disk usage
---

In [ ]:
## Code cell 44 ##

du -h -d2 ${gohome}$USER

We now have a personal folder that gets too heavy.  Let's remove some files we won't use anymore: 

- initial srr files in Data/sra/   
- raw fastq.gz files in Data/fastq/raw/   
- cleaned fastq.gz files in /Results/fastp   
- intermediate Aligned.sortedByNames.bam files produced by <code>samtools</code> for <code>featuresCounts</code> above, if the rm command was masked at the end of Code cell 12.  

In [ ]:
## Code cell 45 ##

# Saving disk space

# Removing:
# initial srr files
rm -r ${gohome}$USER/Data/sra/

# raw fastq.gz
rm ${gohome}$USER/Data/fastq/raw/*.fastq.gz

# cleaned fastq.gz
#rm ${gohome}$USER/Results/fastp/*.fastp.fastq.gz

# intermediate Aligned.sortedByNames.bam
# rm ${gohome}$USER/Results/featurecounts/*_Aligned.sortedByNames.bam    # if is was not already done in cell 11

Let's see our disk usage now:

In [ ]:
## Code cell 46 ##

du -h -d2 ${gohome}$USER

---
___

## Conclusion


**Next Practical session**

Now we go on with an introduction to the R language, that will be used during the next steps of the analysis.  
  
**=> Step 7: Introduction to R** 

The jupyter notebook used for the next session will be *Pipe_07-R403_Intro-to-R.ipynb*    
Let's retrieve it in our personal directory, in order to have a private copy to work on:   

In [ ]:
## Code cell 47 ##   

cp "${gohome}pipeline/Pipe_07a-R_intro-to-R.ipynb" "${gohome}$USER/"
cp "${gohome}alldata/Example_Data/Temperatures.txt" "${gohome}$USER/"



**Save executed notebook**

To end the session, save your executed notebook in your `run_notebooks` folder. Adjust the name with yours and reformat as Code cell to run it.

<div class="alert alert-block alert-success"><b>Success:</b> Well done! You now know how to count reads per features.<br>
Don't forget to save you notebook and export a copy as an <b>html</b> file as well <br>
- Open "File" in the Menu<br>
- Select "Export Notebook As"<br>
- Export notebook as HTML<br>
- You can then open it in your browser even without being connected to the server! 
</div>

---
---

## Useful commands
<div class="alert alert-block alert-info"> 
    
- <kbd>CTRL</kbd>+<kbd>S</kbd> : save notebook<br>    
- <kbd>CTRL</kbd>+<kbd>ENTER</kbd> : Run Cell<br>  
- <kbd>SHIFT</kbd>+<kbd>ENTER</kbd> : Run Cell and Select Next<br>   
- <kbd>ALT</kbd>+<kbd>ENTER</kbd> : Run Cell and Insert Below<br>   
- <kbd>ESC</kbd>+<kbd>y</kbd> : Change to *Code* Cell Type<br>  
- <kbd>ESC</kbd>+<kbd>m</kbd> : Change to *Markdown* Cell Type<br> 
- <kbd>ESC</kbd>+<kbd>r</kbd> : Change to *Raw* Cell Type<br>    
- <kbd>ESC</kbd>+<kbd>a</kbd> : Create Cell Above<br> 
- <kbd>ESC</kbd>+<kbd>b</kbd> : Create Cell Below<br> 

<em>  
To make nice html reports with markdown: <a href="https://dillinger.io/" title="dillinger.io">html visualization tool 1</a> or <a href="https://stackedit.io/app#" title="stackedit.io">html visualization tool 2</a>, <a href="https://www.tablesgenerator.com/markdown_tables" title="tablesgenerator.com">to draw nice tables</a>, and the <a href="https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd" title="Ultimate guide">Ultimate guide</a>. <br>
Further reading on JupyterLab notebooks: <a href="https://jupyterlab.readthedocs.io/en/latest/user/notebook.html" title="Jupyter Lab">Jupyter Lab documentation</a>.<br>   
</em>    
 
</div>

---
Bénédicte Noblet - 05-07 2021   
Sandrine Caburet - 02-05 2023   
Claire Vandiedonck - 03-06 2023  
Maj 16/06/2024 by @CVandiedonck